In [1]:
import numpy as np
import cvxpy as cvx



In [5]:
# KIRCHHOFF
I_NG = np.block([[1, 0],
                 [0, 1],
                 [0, 0]])

I_NL = np.block([[1, 1, 0], 
                 [-1, 0, 1],
                 [0, -1, -1]])

I_ND = np.block([[0], [0], [1]])

# OHM (susceptance)
N_sus = np.block([[1, -1, 0],
                  [1, 0, -1],
                  [0, 1, -1]])

# SHIFT FACTORS
SF_LN = np.block([[0, -2/3, -1/3],
                  [0, -1/3, -2/3],
                  [0, 1/3, -1/3]])

# LIMITS TO POWER FLOW
PL_max = np.block([[150], [150], [150]])

# POWER DEMAND
P_D = np.block([[150]])

# EXTRA
n_G = I_NG.shape[1]
n_D = I_ND.shape[1]
ones_g = np.ones(shape=(n_G, 1))
ones_d = np.ones(shape=(n_D, 1))


In [6]:
# COST
c = np.array([[ 10, 12 ]])

# SUPPLY = DEMAND CONSTRAINT
A_eq = ones_g.T
b_eq = ones_d.T@P_D

# POWER BALANCE CONSTRAINT
A_ub = np.block([[SF_LN@I_NG], 
                 [-SF_LN@I_NG]])
b_ub = np.block([[PL_max + SF_LN@I_ND@P_D],
                 [PL_max - SF_LN@I_ND@P_D]]).squeeze()

# POSITIVITY AND CAPACITY
#P1_bounds = (0, None)
#P2_bounds = (0, None)

In [7]:
# Create optimization variables.
P = cvx.Variable(n_G,1)

# Create two constraints.
constraints = [-A_eq*P == -b_eq,
               A_ub*P <= b_ub,
               P >= 0]

# Form objective.
obj = cvx.Minimize(c*P)

# Form and solve problem.
prob = cvx.Problem(obj, constraints)
prob.solve()

# The optimal dual variable (Lagrange multiplier) for
# a constraint is stored in constraint.dual_value.
print("optimal (A_eq*P == b_eq) dual variable", constraints[0].dual_value)
print("optimal (A_ub*P <= b_ub) dual variable", constraints[1].dual_value)
print("c*P:", prob.value)

optimal (A_eq*P == b_eq) dual variable [10.]
optimal (A_ub*P <= b_ub) dual variable [1.17588632e-10 2.68550510e-10 4.20313786e-10 2.08244432e-10
 2.80784552e-10 3.34873298e-10]
c*P: 1500.000000150917
